## Прогнозирование временных рядов с помощью библиотеки Prophet

### Введение

В официальной документации сказано:

   *Prophet is a procedure for forecasting time series data based on an additive model where non-linear trends are fit with yearly, weekly, and daily seasonality, plus holiday effects. It works best with time series that have strong seasonal effects and several seasons of historical data. Prophet is robust to missing data and shifts in the trend, and typically handles outliers well.*

   *Prophet is open source software released by Facebook’s Core Data Science team. It is available for download on CRAN and PyPI.*


- **Тренд** моделирует непериодические изменения в данных временного ряда.

- **Сезонность** вызвана периодическими изменениями, такими как ежедневная, еженедельная или ежегодная сезонность.

- **Эффект праздников**, который возникает в нерегулярных графиках в течение дня или периода дней.

- **Остатки модели** — это то, что не объясняется моделью.



### Преимущества Prophet


- **1. Точный и быстрый** - Prophet точен и быстр. Он используется во многих приложениях в Facebook для создания надежных прогнозов для планирования и постановки целей. 

- **2. Полностью автоматический** - Prophet полностью автоматический. Мы получим разумный прогноз по запутанным данным без ручного труда. 

- **3. Настраиваемые прогнозы** - Prophet создает настраиваемые прогнозы. Он включает в себя множество возможностей для пользователей настраивать и корректировать прогнозы. Мы можем использовать интерпретируемые человеком параметры для улучшения прогноза, добавляя наши знания о предметной области.

- **4. Доступен в R или Python** - Мы можем реализовать использовать библиотеку Prophet в R или Python. 

- **5. Хорошо обрабатывает сезонные колебания** - Prophet учитывает сезонность с несколькими периодами.

- **6. Устойчив к выбросам** - Он устойчив к выбросам. Он обрабатывает выбросы, удаляя их.

- **7. Устойчив к пропущенным данным** - Prophet устойчив к пропущенным данным.

In [ ]:
from prophet import Prophet
from prophet.plot import plot_plotly
import plotly.offline as py
py.init_notebook_mode()
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('fivethirtyeight')

In [ ]:
file = 'airline-passengers.csv'

df = pd.read_csv(file)

In [ ]:
df.head()

In [ ]:
df.rename(columns = {'#Passengers':'AirPassengers'}, inplace = True)

In [ ]:
df.info()

Надо преобразовывать колонку данных в тип данных datatime

In [ ]:
df['Month'] = pd.DatetimeIndex(df['Month'])
df.dtypes

We can now see that our `Month` column is of the correct datetime type.

Требование библиотеки: колонки в данных должны быть **ds (время)**  и **y (значение)**. 

In [ ]:
df = df.rename(columns={'Month': 'ds','Passengers': 'y'})
df.head()

In [ ]:
ax = df.set_index('ds').plot(figsize=(12, 8))
ax.set_ylabel('Monthly Number of Airline Passengers')
ax.set_xlabel('Date')

plt.show()

Now, our dataset is prepared and we are ready to use the Prophet library to produce forecasts of our time series.

In [ ]:
# set the uncertainty interval to 95% (the Prophet default is 80%)
my_model = Prophet(interval_width=0.95)

In [ ]:
df

In [ ]:
my_model.fit(df)

In [ ]:
future_dates = my_model.make_future_dataframe(periods=36, freq='MS')
future_dates.head()

- In the code snippet above, we instructed Prophet to generate 36 datestamps in the future.

- При работе с Prophet важно учитывать частоту наших временных рядов. 

- Поскольку мы работаем с ежемесячными данными, мы четко указали желаемую частоту временных меток (в данном случае `MS` - это начало месяца). 

- Поэтому `make_future_dataframe` сгенерировал для нас 36 ежемесячных временных меток. 

In [ ]:
forecast = my_model.predict(future_dates)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].head()

Prophet возвращает большой DataFrame со множеством интересных столбцов, но мы подмножествуем наши выходные данные по столбцам, наиболее релевантным для прогнозирования. Это:

- **ds**: метка даты прогнозируемого значения
- **yhat**: прогнозируемое значение нашей метрики (в статистике yhat — это обозначение, традиционно используемое для представления прогнозируемых значений значения y)
- **yhat_lower**: нижняя граница наших прогнозов
- **yhat_upper**: верхняя граница наших прогнозов

In [ ]:
my_model.plot(forecast, uncertainty=True)

- Prophet строит графики наблюдаемых значений нашего временного ряда (черные точки), прогнозируемых значений (синяя линия) и интервалов неопределенности наших прогнозов (области, закрашенные синим цветом).

- Еще одной особенно сильной особенностью Prophet является его способность возвращать компоненты наших прогнозов. 

- Это может помочь выявить, как ежедневные, недельные и годовые закономерности временного ряда влияют на общие прогнозируемые значения.

In [ ]:
my_model.plot_components(forecast)

- Приведенный выше график дает интересные сведения. 

- Первый график показывает, что ежемесячный объем авиапассажиров линейно увеличивается с течением времени. 

- Второй график подчеркивает тот факт, что еженедельное количество пассажиров достигает пика к концу недели и в субботу. 

- Третий график показывает, что наибольший трафик приходится на праздничные месяцы в июле и августе.

In [ ]:
fig1 = my_model.plot_components(forecast)

### Добавление точек изменения в Prophet

- Точки изменения — это точки даты и времени, в которых временной ряд резко меняет траекторию.

- По умолчанию Prophet добавляет 25 точек изменения к начальным 80% набора данных.

- Давайте построим вертикальные линии, где возникали потенциальные точки изменения.

In [ ]:
from prophet.plot import add_changepoints_to_plot
fig = my_model.plot(forecast)
a = add_changepoints_to_plot(fig.gca(), my_model, forecast)

Мы можем просмотреть даты, когда произошли точки изменения.

In [ ]:
my_model.changepoints

- Мы можем изменить предполагаемый диапазон точек изменения, установив *changepoint_range*

In [ ]:
pro_change= Prophet(changepoint_range=0.9)
forecast = pro_change.fit(df).predict(future_dates)
fig= pro_change.plot(forecast);
a = add_changepoints_to_plot(fig.gca(), pro_change, forecast)

Количество точек изменения можно задать с помощью параметра *n_changepoints* при инициализации Prophet.

In [ ]:
pro_change= Prophet(n_changepoints=20, yearly_seasonality=True)
forecast = pro_change.fit(df).predict(future_dates)
fig= pro_change.plot(forecast);
a = add_changepoints_to_plot(fig.gca(), pro_change, forecast)

### Настройка тренда

- Prophet позволяет нам настроить тренд в случае переобучения или недообучения. 

- **changepoint_prior_scale** помогает настроить силу тренда.

- Значение по умолчанию для **changepoint_prior_scale** равно 0,05. 

- Уменьшите значение, чтобы сделать тренд менее гибким. 

- Увеличьте значение changepoint_prior_scale, чтобы сделать тренд более гибким.

- Увеличьте **changepoint_prior_scale** до 0,08, чтобы сделать тренд гибким.

In [ ]:
pro_change= Prophet(n_changepoints=20, yearly_seasonality=True, changepoint_prior_scale=0.08)
forecast = pro_change.fit(df).predict(future_dates)
fig= pro_change.plot(forecast);
a = add_changepoints_to_plot(fig.gca(), pro_change, forecast)

- Уменьшим **changepoint_prior_scale** до 0.001 для того, чтобы тренд был менее "гибким"

In [ ]:
pro_change= Prophet(n_changepoints=20, yearly_seasonality=True, changepoint_prior_scale=0.001)
forecast = pro_change.fit(df).predict(future_dates)
fig= pro_change.plot(forecast);
a = add_changepoints_to_plot(fig.gca(), pro_change, forecast)

### Пример прогнозирования цены акций с помощью Prophet

In [ ]:
stock_price =  pd.read_csv('SBER.csv',parse_dates=['Дата'])[::-1].reset_index(drop=True)

In [ ]:
import datetime
stock_price = stock_price[['Дата','Цена']]
stock_price.columns = ['ds', 'y']
stock_price['y'] = stock_price['y'].apply(lambda x: x.replace(',','.')).astype(float)
stock_price['ds'] = stock_price['ds'].apply(lambda x: datetime.datetime.strptime(x, "%d.%m.%Y").strftime("%Y-%m-%d")).astype('datetime64[ns]')
stock_price

In [ ]:
stock_price.set_index('ds').y.plot(figsize=(12,6), grid=True)
plt.show()

In [ ]:
model = Prophet()
model.fit(stock_price)

Чтобы создать прогноз с помощью нашей модели, нам нужно создать несколько будущих дат. Prophet предоставляет нам вспомогательную функцию make_future_dataframe. Мы передаем количество будущих периодов и частоту. Выше мы создали прогноз на следующие 365 дней или 1 год.

Поскольку акции могут торговаться только по будням, нам нужно удалить выходные из нашего прогнозного фрейма данных. Для этого мы создаем логическое выражение, где, если день не равен 0 - 4, то возвращается False. "0 = понедельник, 6 = суббота и т. д.."

Затем мы передаем логическое выражение в наш фрейм данных, который возвращает только значения True. Теперь у нас есть прогнозный фрейм данных, состоящий из следующего года.

In [ ]:
future = model.make_future_dataframe(365, freq='d')

future_boolean = future['ds'].map(lambda x : True if x.weekday() in range(0, 5) else False)
future = future[future_boolean] 

In [ ]:
forecast = model.predict(future)
model.plot(forecast)
plt.show()

In [ ]:
model.plot_components(forecast)
plt.show()

In [ ]:
stock_price_forecast = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
df = pd.merge(stock_price, stock_price_forecast, on='ds', how='right')
df.set_index('ds').plot(figsize=(16,8), color=['royalblue', "#34495e", "#e74c3c", "#e74c3c"], grid=True)

#### Моделирование прогнозов

Не хочется принимать никаких торговых решений по построенному без бэктестинга производительности и торговой стратегии.

В этом разделе мы будем моделировать, как если бы Prophet существовал в 1980 году, и мы использовали бы его для создания ежемесячного прогноза до 2019 года. Затем мы будем использовать эти данные в следующем разделе, чтобы моделировать, как работали различные торговые стратегии по сравнению с тем, если бы мы просто купили и удерживали фьючерс на индекс S&P500.

In [ ]:
stock_price['dayname'] = stock_price['ds'].dt.day_name()
stock_price['month'] = stock_price['ds'].dt.month
stock_price['year'] = stock_price['ds'].dt.year
stock_price['month/year'] = stock_price['month'].map(str) + '/' + stock_price['year'].map(str) 

stock_price = pd.merge(stock_price, 
                       stock_price['month/year'].drop_duplicates().reset_index(drop=True).reset_index(),
                       on='month/year',
                       how='left')

stock_price = stock_price.rename(columns={'index':'month/year_index'})

In [ ]:
loop_list = stock_price['month/year'].unique().tolist()
max_num = len(loop_list) - 1
forecast_frames = []

for num, item in enumerate(loop_list):

    if  num == max_num:
        pass
    else:
        df = stock_price.set_index('ds')[
             stock_price[stock_price['month/year'] == loop_list[0]]['ds'].min():\
             stock_price[stock_price['month/year'] == item]['ds'].max()]
        
        df = df.reset_index()[['ds', 'y']]
        
        model = Prophet()
        model.fit(df)
        
        future = stock_price[stock_price['month/year_index'] == (num + 1)][['ds']]

        forecast = model.predict(future)
        forecast_frames.append(forecast)

In [ ]:
from functools import reduce
stock_price_forecast = reduce(lambda top, bottom: pd.concat([top, bottom], sort=False), forecast_frames)
stock_price_forecast = stock_price_forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
stock_price_forecast.to_csv('stock_price_forecast.csv', index=False)

In [ ]:
stock_price_forecast = pd.read_csv('stock_price_forecast.csv', parse_dates=['ds'])
df = pd.merge(stock_price[['ds','y', 'month/year_index']], stock_price_forecast, on='ds')
df['Percent Change'] = df['y'].pct_change()
df.set_index('ds')[['y', 'yhat', 'yhat_lower', 'yhat_upper']].plot(figsize=(16,8), color=['royalblue', "#34495e", "#e74c3c", "#e74c3c"], grid=True)
plt.show()

#### Разные стратегии

In [ ]:
df['Hold'] = (df['Percent Change'] + 1).cumprod()
df['Prophet'] = ((df['yhat'].shift(-1) > df['yhat']).shift(1) * (df['Percent Change']) + 1).cumprod()
df['Prophet Thresh']  = ((df['y'] > df['yhat_lower']).shift(1)* (df['Percent Change']) + 1).cumprod()
df['Seasonality'] = ((~df['ds'].dt.month.isin([8,9])).shift(1) * (df['Percent Change']) + 1).cumprod()

Выше мы создаем четыре начальных торговых алгоритма:

**Hold:** наш ориентир. Это стратегия покупки и удержания. Это означает, что мы покупаем акции и держим их до конца периода времени.

**Prophet:** эта стратегия заключается в продаже, когда наш прогноз указывает на нисходящий тренд, и в покупке обратно, когда он указывает на восходящий тренд.

**Prophet Thresh:** эта стратегия заключается в продаже только тогда, когда цена акций падает ниже нашей границы yhat_lower.

**Seasonality:** эта стратегия заключается в выходе с рынка в августе и повторном входе в октябре. Это было основано на графике сезонности выше.

In [ ]:
(df.dropna().set_index('ds')[['Hold', 'Prophet', 'Prophet Thresh','Seasonality']] * 1000).plot(figsize=(16,8), grid=True)

print(f"Hold = {df['Hold'].iloc[-1]*1000:,.0f}")
print(f"Prophet = {df['Prophet'].iloc[-1]*1000:,.0f}")
print(f"Prophet Thresh = {df['Prophet Thresh'].iloc[-1]*1000:,.0f}")
print(f"Seasonality = {df['Seasonality'].iloc[-1]*1000:,.0f}")

In [ ]:
performance = {}

for x in np.linspace(.9,.99,10):
    y = ((df['y'] > df['yhat_lower']*x).shift(1)* (df['Percent Change']) + 1).cumprod()
    performance[round(x,2)] = y
    
best_yhat = pd.DataFrame(performance).max().idxmax()
pd.DataFrame(performance).plot(figsize=(16,8), grid=True, lw = 1)
f'Best Yhat = {best_yhat:,.2f}'

Выше мы проходим через различные проценты порога, чтобы найти оптимальный порог. Похоже, что лучший порог — это 90% от нашего текущего yhat_lower.

In [ ]:
df['Optimized Prophet Thresh']  = ((df['y'] > df['yhat_lower'] * best_yhat).shift(1) * 
                                   (df['Percent Change']) + 1).cumprod()

In [ ]:
(df.dropna().set_index('ds')[['Hold', 'Prophet', 'Prophet Thresh',
                              'Seasonality', 'Optimized Prophet Thresh']] * 1000).plot(figsize=(16,8), grid=True, lw = 1.4)

print(f"Hold = {df['Hold'].iloc[-1]*1000:,.0f}")
print(f"Prophet = {df['Prophet'].iloc[-1]*1000:,.0f}")
print(f"Prophet Thresh = {df['Prophet Thresh'].iloc[-1]*1000:,.0f}")
print(f"Seasonality = {df['Seasonality'].iloc[-1]*1000:,.0f}")
print(f"Optimized Prophet Thresh = {df['Optimized Prophet Thresh'].iloc[-1]*1000:,.0f}")

Будем создать Optimized Thresh для каждого текущего момента времени нашего прогноза.

In [ ]:
fcst_thresh = {}

for num, index in enumerate(df['month/year_index'].unique()):

    temp_df = df.set_index('ds')[
         df[df['month/year_index'] == df['month/year_index'].unique()[0]]['ds'].min():\
         df[df['month/year_index'] == index]['ds'].max()]

    performance = {}
    
    for thresh in np.linspace(0, .99, 100):
        percent =  ((temp_df['y'] > temp_df['yhat_lower'] * thresh).shift(1)* (temp_df['Percent Change']) + 1).cumprod()
        performance[thresh] = percent
    
    best_thresh = pd.DataFrame(performance).max().idxmax()
    
    if num == len(df['month/year_index'].unique())-1:
        pass
    else:
        fcst_thresh[df['month/year_index'].unique()[num+1]] = best_thresh

In [ ]:
fcst_thresh = pd.DataFrame([fcst_thresh]).T.reset_index().rename(columns={'index':'month/year_index', 0:'Fcst Thresh'})

In [ ]:
fcst_thresh['Fcst Thresh'].plot(figsize=(16,8), grid=True);

Выше, как и в случае, когда мы создавали наш ежемесячный прогноз, мы проходим по данным и находим оптимальный период процента порога на сегодняшний день для текущего момента времени. Как вы можете видеть, % текущего порога скачет по мере того, как мы идем глубже в исторические данные

In [ ]:
df['yhat_optimized'] = pd.merge(df, fcst_thresh, 
                                on='month/year_index', 
                                how='left')['Fcst Thresh'].shift(1) * df['yhat_lower']

In [ ]:
df['Prophet Fcst Thresh']  = ((df['y'] > df['yhat_optimized']).shift(1)* (df['Percent Change']) + 1).cumprod()

In [ ]:
(df.dropna().set_index('ds')[['Hold', 'Prophet', 'Prophet Thresh',
                              'Prophet Fcst Thresh']] * 1000).plot(figsize=(16,8), grid=True)

print(f"Hold = {df['Hold'].iloc[-1]*1000:,.0f}")
print(f"Prophet = {df['Prophet'].iloc[-1]*1000:,.0f}")
print(f"Prophet Thresh = {df['Prophet Thresh'].iloc[-1]*1000:,.0f}")
# print(f"Seasonality = {df['Seasonality'].iloc[-1]*1000:,.0f}")
print(f"Prophet Fcst Thresh = {df['Prophet Fcst Thresh'].iloc[-1]*1000:,.0f}")